# Imports

In [1]:
##########
# basics #
##########

import warnings
warnings.filterwarnings('ignore')
import datetime
import glob
import hashlib
import itertools
import math
import operator
import os
import pickle
import random
import re
import string
import sys
import time

###########
# science #
###########

import scipy as sp
import numpy as np
import pandas as pd
rseed = random.seed(42)

#################
# visualization #
#################

# matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
mpl.rcParams['figure.figsize']=(12.0,4.0)
%matplotlib inline

############
# sys info #
############

%reload_ext watermark
%watermark -a "Ken Cavagnolo" -n -u -v -m -h -g -p numpy,scipy,pandas,\
matplotlib,seaborn,plotly,boto3

Ken Cavagnolo 
last updated: Mon Nov 28 2016 

CPython 2.7.10
IPython 5.1.0

numpy 1.11.2
scipy 0.18.1
pandas 0.19.0
matplotlib 1.5.3
seaborn 0.8.dev0
plotly 1.12.9
boto3 1.4.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.0.72)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
host name  : DrGonzo.local
Git hash   : 5fc097e261dd62154fcb1342710b3931297c54c0


In [2]:
import boto3

In [3]:
AWS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY']

# Setup S3

In [4]:
bucket_name = ''.join(random.choice(string.ascii_lowercase+string.digits) for i in range(32))
print bucket_name

xaji0y6dpbhsahxthv3a3zmf8mdd4v30


In [6]:
s3_client = boto3.client('s3')
s3_client.create_bucket(Bucket=bucket_name)
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

In [7]:
trainfile = '../datasets/banking.csv'
testfile = '../datasets/banking-batch.csv'

In [8]:
s3.meta.client.upload_file(trainfile, bucket_name, 'train.csv')

In [9]:
s3.meta.client.upload_file(testfile, bucket_name, 'test.csv')

In [10]:
for obj in bucket.objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='xaji0y6dpbhsahxthv3a3zmf8mdd4v30', key=u'test.csv')
s3.ObjectSummary(bucket_name='xaji0y6dpbhsahxthv3a3zmf8mdd4v30', key=u'train.csv')


# Setup ML

In [11]:
ml_client = boto3.client('machinelearning')

In [14]:
ml_client.create_data_source_from_s3(
    DataSourceId='bank_train',
    DataSourceName='banking_data_1',
    DataSpec={
        'DataLocationS3': 's3://'+bucket_name+'/train.csv',
    },
    ComputeStatistics=True
)

ClientError: An error occurred (InvalidInputException) when calling the CreateDataSourceFromS3 operation: FAILURES (1): Exactly one of the following parameters should be provided, but they are all blank: data schema, data schema URI.

# Clean-up

In [15]:
for obj in bucket.objects.all():
    s3.Object(bucket.name, obj.key).delete()

In [16]:
bucket.delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Tue, 29 Nov 2016 04:49:25 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': '5KF8PydE9rOuV+VfjJcreoawVoZEckCCRfTJdCN8USlWgRi1HbNI+0rWAoOLgadIstHtaz042UQ=',
   'x-amz-request-id': 'DF65A9664ED2BE59'},
  'HTTPStatusCode': 204,
  'HostId': '5KF8PydE9rOuV+VfjJcreoawVoZEckCCRfTJdCN8USlWgRi1HbNI+0rWAoOLgadIstHtaz042UQ=',
  'RequestId': 'DF65A9664ED2BE59',
  'RetryAttempts': 0}}